# Mathematics for AI - Coursework Task 2

The second task is about classifying handwritten digits. We will use the MNIST dataset for training and testing. The point of this task is to develop a multi-layer neural network for classificationusing mostly Numpy:

• Implement sigmoid and relu layers (with forward and backward pass)

• Implement a softmax output layer

• Implement a fully parameterizable neural network (number and types of layers, number of units)

• Implement an optimizer(e.g. SGD or Adam)and a stopping criterionof your choosing
• Train your Neural Network using backpropagation. Evaluate different neural network architectures andcompare your different results. You can also compare withthe results presented inhttp://yann.lecun.com/exdb/mnist/

In [48]:
# Import limited libraries
from sklearn.datasets import load_digits
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

In [49]:
# Load MNIST dataset
df = load_digits()

# Divorce data and target 
X = df.data
y = df.target

In [50]:
# Splitting Dataset - Inspiration taken from Lab 04 
ratio = 0.75
length = len(X)

X_train = X[:int(length*ratio)]
X_test = X[int(length*ratio):]
y_train = y[:(length - int(length*ratio))]
y_test = y[(length - int(length*ratio)):]

In [ ]:
# Normalize Dataset

X_train = X_train / 255.0
X_test = X_test / 255.0

https://towardsdatascience.com/lets-code-a-neural-network-in-plain-numpy-ae7e74410795

In [52]:
# Mathematical functions

# Sigmoid function
    
def Sigmoid_forwards(inputs):
    return 1/(1+np.exp(-inputs))

    # Derivative of Sigmoid (for backward propagation) 
def Sigmoid_backwards(derivative, inputs):
    sig = sigmoid_forwards(inputs)
    return derivative * sig * (1 - sig)

# ReLu function

def Relu_forward(inputs):
    return np.maximum(0, inputs)

# Derivative of ReLu
def Relu_backward(derivative, inputs):
    derivative = np.array(derivative, copy = True)
    derivative[inputs <= 0] = 0;
    return derivative;

# Softmax function for output
def Softmax_forward(self):
    e = exp(self.inputs)
    return e / e.sum()

# Derivative of softmax
def Softmax_backward(derivative, inputs):
    soft = softmax(inputs)
    return derivative * soft * (1 - soft)

In [53]:
# Cost function
def get_cost(target, output):
    m = target.shape[1]
    cost = -1 / m * (np.dot(output, np.log(target).T) + np.dot(1 - output, np.log(1 - target).T))
    return np.squeeze(cost)

In [ ]:
np.random.seed(0)
class NeuralNetwork:
    
    def __init__(self, n_inputs, n_neurons, n_layers, n_outputs):
        
        self.n_inputs = n_inputs
        self.n_neurons = n_neurons
        self.n_layers = n_layers
        self.n_outputs = self.n_outputs
        self.weights = 0.1*np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeroes((1, n_neurons))
        
    def forward(self, inputs, activation)
    
        output_forward = np.dot(inputs, self.weights) + self.biases
        
        if(activation == 'sigmoid')
            activation_function = Sigmoid_forward()
        elif(activation == 'relu')
            activation_function = Relu_forward()
        elif(activation == 'softmax')
            activation_function = Softmax_forward()
        else:
            raise Exception('Activation function not defined')
            
        return activation_function.forward(self.output_forward), self.output_forward
    
    def forward_full(self)
        memory = {}
        
        for i in range self.n_layers
            layer_number = i + 1
            
            
            result, output = self.forward
    
    def backward(self, inputs):
        m = inputs.shape
    
        if(activation == 'sigmoid')
            activation_function = Sigmoid_backward()
        elif(activation == 'relu')
            activation_function = Relu_backward()
        elif(activation == 'softmax')
            activation_function = Softmax_backward()
        else:
            raise Exception('Activation function not defined')
            
        b_activation = activation_function()
        b_weight = np.dot().T / m
        b_bias = np.sum() / m
        b_output
        
    def update_weight(self)
        